<a href="https://colab.research.google.com/github/RehanKhn/Boosting-Buddy-A-Conversational-Agent-for-Improvement-of-Mental-Health/blob/main/IBDM_Movie_Review_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':'1sTFwYuZ_IyyunrKOeKxH4QCGn7xw0D-d'}) 
downloaded.GetContentFile('reviews.csv')

In [ ]:
data = pd.read_csv("reviews.csv")
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [ ]:
data['review'] = data['review'].str.lower()

In [ ]:
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", 
             "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
             "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", 
             "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into",
             "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or",
             "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", 
             "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's",
             "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up",
             "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's",
             "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've",
             "your", "yours", "yourself", "yourselves" ]

In [ ]:
def remove_stopwords(data):
  data['review without stopwords'] = data['review'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
  return data

def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result
    
data_without_stopwords = remove_stopwords(data)
data_without_stopwords['clean_review']= data_without_stopwords['review without stopwords'].apply(lambda cw : remove_tags(cw))
data_without_stopwords['clean_review'] = data_without_stopwords['clean_review'].str.replace('[{}]'.format(string.punctuation), ' ')


In [ ]:
reviews_list = []
reviews = data['review']
for i in range(len(reviews)):
  reviews_list.append(reviews[i])
sentiment = data_without_stopwords['sentiment']
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, sentiment)))

X_train, X_test,Y_train, Y_test = train_test_split(reviews_list, y, test_size=0.2, random_state = 45)


In [ ]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

words_to_index = tokenizer.word_index

In [ ]:
downloaded = drive.CreateFile({'id':'1QUlCL5MATjZXR8rZrFG4ZvKXXKNOGb5o'}) 
downloaded.GetContentFile('glove6B50d.txt')

In [ ]:

with open("glove6B50d.txt", 'r', encoding='UTF-8') as f:
  words = set()
  word_to_vec_map = {}
  for line in f:
    w_line = line.split()
    curr_word = w_line[0]
    word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)

maxLen = 150

In [ ]:

vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['moon'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None:
    emb_matrix[index, :] = embedding_vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)





In [ ]:
def imdb_rating(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = LSTM(128, return_sequences=True)(embeddings)

  X = Dropout(0.6)(X)

  X = LSTM(128, return_sequences=True)(X)

  X = Dropout(0.6)(X)

  X = LSTM(128)(X)

  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model

In [ ]:
def conv1d_model(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = Conv1D(512,3,activation='relu')(embeddings)
  
  X = MaxPooling1D(3)(X)

  X = Conv1D(256,3,activation='relu')(X)
  
  X = MaxPooling1D(3)(X)

  X = Conv1D(256,3,activation='relu')(X)
  X = Dropout(0.8)(X)
  X = MaxPooling1D(3)(X)

  X = GlobalMaxPooling1D()(X)

  X = Dense(256, activation='relu')(X)
  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model

In [ ]:
model = imdb_rating((maxLen,))
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 150)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 50)           5612100   
_________________________________________________________________
lstm_3 (LSTM)                (None, 150, 128)          91648     
_________________________________________________________________
dropout_2 (Dropout)          (None, 150, 128)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 150, 128)          131584    
_________________________________________________________________
dropout_3 (Dropout)          (None, 150, 128)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               1315

In [ ]:
model_1d = conv1d_model((maxLen,))
model_1d.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 150)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 50)           5612100   
_________________________________________________________________
conv1d (Conv1D)              (None, 148, 512)          77312     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 49, 512)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 47, 256)           393472    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 15, 256)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 13, 256)           1968

In [ ]:
X_train_indices = tokenizer.texts_to_sequences(X_train)

In [ ]:
X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')
X_train_indices.shape

(40000, 150)

In [ ]:
adam = keras.optimizers.Adam(learning_rate = 0.0001)
model_1d.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_1d.fit(X_train_indices, Y_train, batch_size=64, epochs=15)

Epoch 1/15
625/625 [==============================] - 12s 8ms/step - loss: 0.6312 - accuracy: 0.6312
Epoch 2/15
625/625 [==============================] - 5s 8ms/step - loss: 0.4972 - accuracy: 0.7564
Epoch 3/15
625/625 [==============================] - 5s 8ms/step - loss: 0.4489 - accuracy: 0.7895
Epoch 4/15
625/625 [==============================] - 5s 8ms/step - loss: 0.4142 - accuracy: 0.8104
Epoch 5/15
625/625 [==============================] - 5s 8ms/step - loss: 0.3887 - accuracy: 0.8236
Epoch 6/15
625/625 [==============================] - 5s 8ms/step - loss: 0.3697 - accuracy: 0.8343
Epoch 7/15
625/625 [==============================] - 5s 8ms/step - loss: 0.3456 - accuracy: 0.8487
Epoch 8/15
625/625 [==============================] - 5s 8ms/step - loss: 0.3257 - accuracy: 0.8573
Epoch 9/15
625/625 [==============================] - 5s 8ms/step - loss: 0.3000 - accuracy: 0.8712
Epoch 10/15
625/625 [==============================] - 5s 8ms/step - loss: 0.2790 - accuracy: 0.880

In [ ]:
adam = keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train_indices, Y_train, batch_size=64, epochs=15)

Epoch 1/15
625/625 [==============================] - 18s 25ms/step - loss: 0.6559 - accuracy: 0.5945
Epoch 2/15
625/625 [==============================] - 15s 25ms/step - loss: 0.5310 - accuracy: 0.7380
Epoch 3/15
625/625 [==============================] - 15s 25ms/step - loss: 0.4852 - accuracy: 0.7685
Epoch 4/15
625/625 [==============================] - 15s 25ms/step - loss: 0.4596 - accuracy: 0.7819
Epoch 5/15
625/625 [==============================] - 15s 25ms/step - loss: 0.4436 - accuracy: 0.7949
Epoch 6/15
625/625 [==============================] - 15s 24ms/step - loss: 0.4357 - accuracy: 0.7977
Epoch 7/15
625/625 [==============================] - 15s 24ms/step - loss: 0.4105 - accuracy: 0.8157
Epoch 8/15
625/625 [==============================] - 15s 25ms/step - loss: 0.4144 - accuracy: 0.8085
Epoch 9/15
625/625 [==============================] - 15s 25ms/step - loss: 0.3958 - accuracy: 0.8221
Epoch 10/15
625/625 [==============================] - 15s 25ms/step - loss: 0.388

In [ ]:
X_test_indices = tokenizer.texts_to_sequences(X_test)

X_test_indices = pad_sequences(X_test_indices, maxlen=maxLen, padding='post')

In [ ]:
model.evaluate(X_test_indices, Y_test)

313/313 [==============================] - 4s 10ms/step - loss: 0.3657 - accuracy: 0.8393


[0.3656839430332184, 0.8392999768257141]

In [ ]:
model_1d.evaluate(X_test_indices, Y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.3794 - accuracy: 0.8344


[0.37937217950820923, 0.8343999981880188]

In [ ]:
preds = model_1d.predict(X_test_indices)

In [ ]:
n = np.random.randint(0,9999)

X_test[n]

'arthur miller has always been known as one of america\'s great playwrights for works such as "death of a salesman" and "the crucible". "focus" is one of his lesser known plays brought to the silver screen. however, knowing what a great playwright arthur miller is, i doubt that his original play was very much like the movie. the movie comes across as empty and formulaic, with william h. macy as a non-jew mistaken for a jew by anti-semitic neighbors in wwii brooklyn. don\'t get me wrong: the acting is ok, and i presume that the people behind this movie were probably trying to make a point about racism, but the movie just doesn\'t work. macy, laura dern and david paymer just can\'t create an effective story with the material here.'

In [ ]:
if preds[n] > 0.5:
  print('predicted sentiment : positive')
else: 
  print('precicted sentiment : negative')

if (Y_test[n] == 1):
  print('correct sentiment : positive')
else:
  print('correct sentiment : negative')

precicted sentiment : negative
correct sentiment : negative


In [ ]:
preds[n]

array([0.46976686], dtype=float32)

In [ ]:
Y_test[n]

0

In [ ]:
reviews_list_idx = tokenizer.texts_to_sequences(reviews_list)

In [ ]:
def add_score_predictions(data, reviews_list_idx):

  data['sentiment score'] = 0

  reviews_list_idx = pad_sequences(reviews_list_idx, maxlen=maxLen, padding='post')

  review_preds = model.predict(reviews_list_idx)

  data['sentiment score'] = review_preds

  pred_sentiment = np.array(list(map(lambda x : 'positive' if x > 0.5 else 'negative',review_preds)))

  data['predicted sentiment'] = 0

  data['predicted sentiment'] = pred_sentiment

  return data

In [ ]:
data = add_score_predictions(data, reviews_list_idx)

In [ ]:
data

,review,sentiment,review without stopwords,clean_review,sentiment score,predicted sentiment
0,one of the other reviewers has mentioned that ...,positive,one reviewers mentioned watching just 1 oz epi...,one reviewers mentioned watching just 1 oz epi...,0.785530,positive
1,a wonderful little production. <br /><br />the...,positive,wonderful little production. <br /><br />the f...,wonderful little production the filming techn...,0.991308,positive
2,i thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...,thought wonderful way spend time hot summer we...,0.834875,positive
3,basically there's a family where a little boy ...,negative,basically family little boy (jake) thinks zomb...,basically family little boy jake thinks zomb...,0.061804,negative
4,"petter mattei's ""love in the time of money"" is...",positive,"petter mattei's ""love time money"" visually stu...",petter mattei s love time money visually stu...,0.989072,positive
...,...,...,...,...,...,...
49995,i thought this movie did a down right good job...,positive,thought movie right good job. wasn't creative ...,thought movie right good job wasn t creative ...,0.952834,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative,"bad plot, bad dialogue, bad acting, idiotic di...",bad plot bad dialogue bad acting idiotic di...,0.039580,negative
49997,i am a catholic taught in parochial elementary...,negative,catholic taught parochial elementary schools n...,catholic taught parochial elementary schools n...,0.443552,negative
49998,i'm going to have to disagree with the previou...,negative,going disagree previous comment side maltin on...,going disagree previous comment side maltin on...,0.489558,negative
